In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 150000) #important for getting all the text
pd.set_option('display.max_columns', 999)
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from IPython.display import Image  
import pydotplus
import random

import scipy
import math
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import warnings

In [2]:
movies= pd.read_csv('/Users/judyliu/Desktop/IEOR 243/Popcron picks/Module 1/ml-latest-small/movies.csv')

# User ID

In [4]:
interactions = pd.read_csv('/Users/judyliu/Desktop/IEOR 243/Popcron picks/Module 1/ml-latest-small/Ratings.csv')

In [5]:
interactions.drop(columns=['timestamp'], inplace=True)
interactions

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [6]:
from sklearn.model_selection import train_test_split
interactions_train_df, interactions_test_df = train_test_split(interactions,
                                   stratify=interactions['userId'], 
                                   test_size=0.20,
                                   random_state=123)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 80668
# interactions on Test set: 20168


In [7]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions.set_index('userId')
interactions_train_indexed_df = interactions_train_df.set_index('userId')
interactions_test_indexed_df = interactions_test_df.set_index('userId')

In [8]:
# Return the list of movie_is that a specific user rates
def get_items_interacted(user_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[user_id]['movieId']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

# Evaluation

In [9]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 50

class ModelEvaluator:


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(movie['movieId'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        
        #Supress depreciation warning. random.sample will lose support for Sets, should be convereted to a list or tuple instead
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            non_interacted_items_sample = random.sample(non_interacted_items, sample_size) 
        
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset['movieId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['movieId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['movieId'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    interactions_train_indexed_df), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['movieId'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['movieId'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()    

In [37]:
class AssociationRuleRecommender:
    MODEL_NAME = 'Association Rule'

    def __init__(self, interactions_df, items_df=None):
        self.interactions_df = interactions_df
        self.items_df = items_df
        self.apriori_result = None

    def get_model_name(self):
        return self.MODEL_NAME

    def generate_association_rules(self, min_support=0.01, min_threshold=1):
        # Convert transactions to one-hot encoded format
        one_hot_encoded = pd.get_dummies(self.interactions_df, columns=['movieId'])
        one_hot_encoded = one_hot_encoded.drop(columns=['rating'])
        
        # Group transactions by user and create a list of movieIds for each user
        grouped = self.interactions_df.groupby('userId')['movieId'].sum()
        new_column_names = {col: str(i) for i, col in enumerate(grouped.columns, start=1)}
        grouped.rename(columns=new_column_names, inplace=True)
        
        # Apply Apriori algorithm to find frequent itemsets
        frequent_itemsets = apriori(grouped.tolist(), min_support=min_support, use_colnames=True)
        
        # Generate association rules
        self.apriori_result = association_rules(frequent_itemsets, metric='lift', min_threshold=min_threshold)

    def recommend_items(self, user_id, items_to_ignore=[], topn=10):
        if self.apriori_result is None:
            raise ValueError("Association rules have not been generated. Call generate_association_rules first.")
        
        # Find items user has interacted with
        user_items = self.interactions_df[self.interactions_df['userId'] == user_id]['movieId'].unique()
        
        # Filter association rules for items the user has interacted with
        relevant_rules = self.apriori_result[self.apriori_result['antecedents'].apply(lambda x: any(item in x for item in user_items))]
        
        # Get recommended items from association rules
        recommendations = relevant_rules.sort_values(by=['confidence', 'lift'], ascending=False)
        
        # Remove items user has seen in the training set
        recommendations = recommendations[~recommendations['consequents'].apply(lambda x: any(item in x for item in user_items))]
        
        # Remove items to ignore
        recommendations = recommendations[~recommendations['consequents'].apply(lambda x: any(item in x for item in items_to_ignore))]
        
        # Select top N recommendations
        recommendations = recommendations.head(topn)
        
        return recommendations

In [10]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [11]:
one_hot_encoded = pd.get_dummies(interactions, columns=['movieId'])

In [12]:
one_hot_encoded = one_hot_encoded.drop(columns=['rating'])

In [13]:
one_hot_encoded 

userId  movieId_1  movieId_2  movieId_3  movieId_4  movieId_5  \
0            1          1          0          0          0          0   
1            1          0          0          1          0          0   
2            1          0          0          0          0          0   
3            1          0          0          0          0          0   
4            1          0          0          0          0          0   
...        ...        ...        ...        ...        ...        ...   
100831     610          0          0          0          0          0   
100832     610          0          0          0          0          0   
100833     610          0          0          0          0          0   
100834     610          0          0          0          0          0   
100835     610          0          0          0          0          0   

        movieId_6  movieId_7  movieId_8  movieId_9  movieId_10  movieId_11  \
0               0          0          0          0           0           0   
1               0          0          0          0           0           0   
2               1          0          0          0           0           0   
3               0          0          0          0           0           0   
4               0          0          0          0           0           0   
...           ...        ...        ...        ...         ...         ...   
100831          0          0          0          0           0           0   
100832          0          0          0          0           0           0   
100833          0          0          0          0           0           0   
100834          0          0          0          0           0           0   
100835          0          0          0          0           0           0   

        movieId_12  movieId_13  movieId_14  movieId_15  movieId_16  \
0                0           0           0           0           0   
1                0           0           0           0           0   
2                0           0           0           0           0   
3                0           0           0           0           0   
4                0           0           0           0           0   
...            ...         ...         ...         ...         ...   
100831           0           0           0           0           0   
100832           0           0           0           0           0   
100833           0           0           0           0           0   
100834           0           0           0           0           0   
100835           0           0           0           0           0   

        movieId_17  movieId_18  movieId_19  movieId_20  movieId_21  \
0                0           0           0           0           0   
1                0           0           0           0           0   
2                0           0           0           0           0   
3                0           0           0           0           0   
4                0           0           0           0           0   
...            ...         ...         ...         ...         ...   
100831           0           0           0           0           0   
100832           0           0           0           0           0   
100833           0           0           0           0           0   
100834           0           0           0           0           0   
100835           0           0           0           0           0   

        movieId_22  movieId_23  movieId_24  movieId_25  movieId_26  \
0                0           0           0           0           0   
1                0           0           0           0           0   
2                0           0           0           0           0   
3                0           0           0           0           0   
4                0           0           0           0           0   
...            ...         ...         ...         ...         ...   
100831           0    

In [14]:
grouped=one_hot_encoded.groupby('userId').sum()

In [15]:
grouped

,movieId_1,movieId_2,movieId_3,movieId_4,movieId_5,movieId_6,movieId_7,movieId_8,movieId_9,movieId_10,movieId_11,movieId_12,movieId_13,movieId_14,movieId_15,movieId_16,movieId_17,movieId_18,movieId_19,movieId_20,movieId_21,movieId_22,movieId_23,movieId_24,movieId_25,movieId_26,movieId_27,movieId_28,movieId_29,movieId_30,movieId_31,movieId_32,movieId_34,movieId_36,movieId_38,movieId_39,movieId_40,movieId_41,movieId_42,movieId_43,movieId_44,movieId_45,movieId_46,movieId_47,movieId_48,movieId_49,movieId_50,movieId_52,movieId_53,movieId_54,movieId_55,movieId_57,movieId_58,movieId_60,movieId_61,movieId_62,movieId_63,movieId_64,movieId_65,movieId_66,movieId_68,movieId_69,movieId_70,movieId_71,movieId_72,movieId_73,movieId_74,movieId_75,movieId_76,movieId_77,movieId_78,movieId_79,movieId_80,movieId_81,movieId_82,movieId_83,movieId_85,movieId_86,movieId_87,movieId_88,movieId_89,movieId_92,movieId_93,movieId_94,movieId_95,movieId_96,movieId_97,movieId_99,movieId_100,movieId_101,movieId_102,movieId_103,movieId_104,movieId_105,movieId_106,movieId_107,movieId_108,movieId_110,movieId_111,movieId_112,movieId_113,movieId_116,movieId_117,movieId_118,movieId_119,movieId_121,movieId_122,movieId_123,movieId_125,movieId_126,movieId_128,movieId_129,movieId_132,movieId_135,movieId_137,movieId_140,movieId_141,movieId_144,movieId_145,movieId_146,movieId_147,movieId_148,movieId_149,movieId_150,movieId_151,movieId_152,movieId_153,movieId_154,movieId_155,movieId_156,movieId_157,movieId_158,movieId_159,movieId_160,movieId_161,movieId_162,movieId_163,movieId_164,movieId_165,movieId_166,movieId_168,movieId_169,movieId_170,movieId_171,movieId_172,movieId_173,movieId_174,movieId_175,movieId_176,movieId_177,movieId_178,movieId_179,movieId_180,movieId_181,movieId_183,movieId_184,movieId_185,movieId_186,movieId_187,movieId_188,movieId_189,movieId_190,movieId_191,movieId_193,movieId_194,movieId_195,movieId_196,movieId_198,movieId_199,movieId_201,movieId_202,movieId_203,movieId_204,movieId_205,movieId_206,movieId_207,movieId_208,movieId_209,movieId_210,movieId_211,movieId_212,movieId_213,movieId_214,movieId_215,movieId_216,movieId_217,movieId_218,movieId_219,movieId_220,movieId_222,movieId_223,movieId_224,movieId_225,movieId_227,movieId_228,movieId_229,movieId_230,movieId_231,movieId_232,movieId_233,movieId_234,movieId_235,movieId_236,movieId_237,movieId_238,movieId_239,movieId_240,movieId_241,movieId_242,movieId_243,movieId_246,movieId_247,movieId_248,movieId_249,movieId_250,movieId_251,movieId_252,movieId_253,movieId_254,movieId_255,movieId_256,movieId_257,movieId_258,movieId_259,movieId_260,movieId_261,movieId_262,movieId_263,movieId_265,movieId_266,movieId_267,movieId_269,movieId_270,movieId_271,movieId_272,movieId_273,movieId_274,movieId_275,movieId_276,movieId_277,movieId_278,movieId_279,movieId_280,movieId_281,movieId_282,movieId_283,movieId_284,movieId_285,movieId_287,movieId_288,movieId_289,movieId_290,movieId_291,movieId_292,movieId_293,movieId_294,movieId_295,movieId_296,movieId_298,movieId_299,movieId_300,movieId_301,movieId_302,movieId_303,movieId_304,movieId_305,movieId_306,movieId_307,movieId_308,movieId_310,movieId_311,movieId_312,movieId_313,movieId_314,movieId_315,movieId_316,movieId_317,movieId_318,movieId_319,movieId_320,movieId_321,movieId_322,movieId_324,movieId_325,movieId_326,movieId_327,movieId_328,movieId_329,movieId_330,movieId_331,movieId_332,movieId_333,movieId_334,movieId_335,movieId_336,movieId_337,movieId_338,movieId_339,movieId_340,movieId_341,movieId_342,movieId_343,movieId_344,movieId_345,movieId_346,movieId_347,movieId_348,movieId_349,movieId_350,movieId_351,movieId_352,movieId_353,movieId_354,movieId_355,movieId_356,movieId_357,movieId_358,movieId_359,movieId_360,movieId_361,movieId_362,movieId_363,movieId_364,movieId_365,movieId_366,movieId_367,movieId_368,movieId_369,movieId_370,movieId_371,movieId_372,movieId_373,movieId_374,movieId_376,movieId_377,movieId_378,movieId_379,movieId_380,movieId_381,movieId_382,movieId_383,movi

In [16]:
new_column_names = {col: str(i) for i, col in enumerate(grouped.columns, start=1)}
grouped.rename(columns=new_column_names, inplace=True)

In [17]:
grouped

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,...,9226,9227,9228,9229,9230,9231,9232,9233,9234,9235,9236,9237,9238,9239,9240,9241,9242,9243,9244,9245,9246,9247,9248,9249,9250,9251,9252,9253,9254,9255,9256,9257,9258,9259,9260,9261,9262,9263,9264,9265,9266,9267,9268,9269,9270,9271,9272,9273,9274,9275,9276,9277,9278,9279,9280,9281,9282,9283,9284,9285,9286,9287,9288,9289,9290,9291,9292,9293,9294,9295,9296,9297,9298,9299,9300,9301,9302,9303,9304,9305,9306,9307,9308,9309,9310,9311,9312,9313,9314,9315,9316,9317,9318,9319,9320,9321,9322,9323,9324,9325,9326,9327,9328,9329,9330,9331,9332,9333,9334,9335,9336,9337,9338,9339,9340,9341,9342,9343,9344,9345,9346,9347,9348,9349,9350,9351,9352,9353,9354,9355,9356,9357,9358,9359,9360,9361,9362,9363,9364,9365,9366,9367,9368,9369,9370,9371,9372,9373,9374,9375,9376,9377,9378,9379,9380,9381,9382,9383,9384,9385,9386,9387,9388,9389,9390,9391,9392,9393,9394,9395,9396,9397,9398,9399,9400,9401,9402,9403,9404,9405,9406,9407,9408,9409,9410,9411,9412,9413,9414,9415,9416,9417,9418,9419,9420,9421,9422,9423,9424,9425,9426,9427,9428,9429,9430,9431,9432,9433,9434,9435,9436,9437,9438,9439,9440,9441,9442,9443,9444,9445,9446,9447,9448,9449,9450,9451,9452,9453,9454,9455,9456,9457,9458,9459,9460,9461,9462,9463,9464,9465,9466,9467,9468,9469,9470,9471,9472,9473,9474,9475,9476,9477,9478,9479,9480,9481,9482,9483,9484,9485,9486,9487,9488,9489,9490,9491,9492,9493,9494,9495,9496,9497,9498,9499,9500,9501,9502,9503,9504,9505,9506,9507,9508,9509,9510,9511,9512,9513,9514,9515,9516,9517,9518,9519,9520,9521,9522,9523,9524,9525,9526,9527,9528,9529,9530,9531,9532,9533,9534,9535,9536,9537,9538,9539,9540,9541,9542,9543,9544,9545,9546,9547,9548,9549,9550,9551,9552,9553,9554,9555,9556,9557,9558,9559,9560,9561,9562,9563,9564,9565,9566,9567,9568,9569,9570,9571,9572,9573,9574,9575,9576,9577,9578,9579,9580,9581,9582,9583,9584,9585,9586,9587,9588,9589,9590,9591,9592,9593,9594,9595,9596,9597,9598,9599,9600,9601,9602,9603,9604,9605,9606,9607,9608,9609,9610,9611,9612,9613,9614,9615,9616,9617,9618,9619,9620,9621,9622,9623,9624,9625,9626,9627,9628,9629,9630,9631,9632,9633,9634,9635,9636,9637,9638,9639,9640,9641,9642,9643,9644,9645,9646,96

In [18]:
from mlxtend.frequent_patterns import apriori, association_rules
frequent_itemsets = apriori(grouped, min_support=0.2, use_colnames=True)

/opt/anaconda3/envs/CIS242/lib/python3.8/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [19]:
frequent_itemsets

/opt/anaconda3/envs/CIS242/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.352459,(1)
1,0.216393,(10)
2,0.290164,(32)
3,0.209836,(33)
4,0.332787,(44)
...,...,...
408,0.208197,"(278, 419, 315, 258)"
409,0.221311,"(511, 278, 315, 258)"
410,0.200000,"(419, 315, 258, 508)"
411,0.203279,"(2225, 315, 258, 1939)"


In [20]:
#Let's view our interpretation values using the Associan rule function.
df_ar = association_rules(frequent_itemsets, metric = "confidence", min_threshold = 0.6)
df_ar

/opt/anaconda3/envs/CIS242/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(1),(225),0.352459,0.411475,0.219672,0.623256,1.514685,0.074644,1.562133,0.524750
1,(1),(258),0.352459,0.503279,0.231148,0.655814,1.303083,0.053762,1.443177,0.359188
2,(1),(278),0.352459,0.519672,0.224590,0.637209,1.226176,0.041427,1.323981,0.284856
3,(1),(315),0.352459,0.539344,0.252459,0.716279,1.328055,0.062362,1.623623,0.381473
4,(1),(419),0.352459,0.390164,0.216393,0.613953,1.573578,0.078877,1.579696,0.562908
...,...,...,...,...,...,...,...,...,...,...
677,"(4792, 3634)","(1939, 4132)",0.270492,0.244262,0.213115,0.787879,3.225544,0.147044,3.562763,0.945808
678,"(4132, 3634)","(1939, 4792)",0.272131,0.242623,0.213115,0.783133,3.227776,0.147089,3.492350,0.948233
679,(4792),"(1939, 4132, 3634)",0.303279,0.221311,0.213115,0.702703,3.175175,0.145996,2.619225,0.983258
680,(4132),"(1939, 4792, 3634)",0.308197,0.224590,0.213115,0.691489,3.078894,0.143897,2.513397,0.976012


In [21]:
grouped_df = interactions.groupby('userId')['movieId'].apply(list).reset_index(name='movieId')

/opt/anaconda3/envs/CIS242/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
grouped_df

/opt/anaconda3/envs/CIS242/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,userId,movieId
0,1,"[1, 3, 6, 47, 50, 70, 101, 110, 151, 157, 163, 216, 223, 231, 235, 260, 296, 316, 333, 349, 356, 362, 367, 423, 441, 457, 480, 500, 527, 543, 552, 553, 590, 592, 593, 596, 608, 648, 661, 673, 733, 736, 780, 804, 919, 923, 940, 943, 954, 1009, 1023, 1024, 1025, 1029, 1030, 1031, 1032, 1042, 1049, 1060, 1073, 1080, 1089, 1090, 1092, 1097, 1127, 1136, 1196, 1197, 1198, 1206, 1208, 1210, 1213, 1214, 1219, 1220, 1222, 1224, 1226, 1240, 1256, 1258, 1265, 1270, 1275, 1278, 1282, 1291, 1298, 1348, 1377, 1396, 1408, 1445, 1473, 1500, 1517, 1552, ...]"
1,2,"[318, 333, 1704, 3578, 6874, 8798, 46970, 48516, 58559, 60756, 68157, 71535, 74458, 77455, 79132, 80489, 80906, 86345, 89774, 91529, 91658, 99114, 106782, 109487, 112552, 114060, 115713, 122882, 131724]"
2,3,"[31, 527, 647, 688, 720, 849, 914, 1093, 1124, 1263, 1272, 1275, 1302, 1371, 1587, 2018, 2080, 2090, 2105, 2288, 2424, 2851, 3024, 3210, 3703, 3949, 4518, 5048, 5181, 5746, 5764, 5919, 6238, 6835, 7899, 7991, 26409, 70946, 72378]"
3,4,"[21, 32, 45, 47, 52, 58, 106, 125, 126, 162, 171, 176, 190, 215, 222, 232, 235, 247, 260, 265, 296, 319, 342, 345, 348, 351, 357, 368, 417, 441, 450, 457, 475, 492, 509, 538, 539, 553, 588, 593, 595, 599, 608, 648, 708, 759, 800, 892, 898, 899, 902, 904, 908, 910, 912, 914, 919, 920, 930, 937, 1025, 1046, 1057, 1060, 1073, 1077, 1079, 1080, 1084, 1086, 1094, 1103, 1136, 1179, 1183, 1188, 1196, 1197, 1198, 1199, 1203, 1211, 1213, 1219, 1225, 1250, 1259, 1265, 1266, 1279, 1282, 1283, 1288, 1291, 1304, 1391, 1449, 1466, 1500, 1517, ...]"
4,5,"[1, 21, 34, 36, 39, 50, 58, 110, 150, 153, 232, 247, 253, 261, 265, 266, 290, 296, 300, 316, 318, 344, 349, 357, 364, 367, 380, 410, 457, 474, 475, 515, 527, 531, 534, 588, 589, 590, 592, 594, 595, 596, 597, 608]"
...,...,...
605,606,"[1, 7, 11, 15, 17, 18, 19, 28, 29, 32, 36, 46, 47, 50, 58, 68, 70, 73, 80, 82, 92, 105, 110, 111, 140, 147, 154, 156, 157, 159, 168, 172, 175, 194, 198, 202, 207, 215, 223, 224, 225, 229, 231, 232, 235, 236, 260, 261, 263, 265, 266, 282, 288, 293, 296, 303, 305, 306, 307, 308, 318, 321, 337, 345, 356, 357, 360, 362, 365, 367, 382, 412, 427, 431, 444, 446, 454, 461, 469, 475, 480, 497, 500, 506, 508, 509, 515, 527, 531, 541, 551, 552, 563, 573, 579, 583, 587, 589, 590, 592, ...]"
606,607,"[1, 11, 25, 34, 36, 86, 110, 112, 150, 153, 165, 188, 204, 208, 241, 260, 292, 296, 316, 318, 337, 353, 366, 377, 380, 381, 382, 423, 434, 448, 457, 480, 517, 527, 553, 589, 592, 593, 608, 695, 724, 736, 780, 798, 849, 858, 891, 919, 920, 949, 1023, 1036, 1079, 1084, 1094, 1097, 1101, 1196, 1207, 1210, 1213, 1214, 1219, 1240, 1249, 1258, 1259, 1265, 1270, 1282, 1291, 1304, 1307, 1320, 1327, 1339, 1356, 1370, 1374, 1377, 1385, 1387, 1394, 1407, 1518, 1527, 1544, 1552, 1580, 1610, 1625, 1717, 1917, 1918, 1954, 1968, 1974, 1975, 1982, 1994, ...]"
607,608,"[1, 2, 3, 10, 16, 19, 21, 24, 31, 32, 34, 39, 44, 47, 48, 50, 63, 65, 70, 88, 93, 95, 104, 107, 110, 111, 141, 145, 150, 153, 158, 160, 163, 168, 170, 172, 173, 174, 180, 181, 185, 188, 193, 196, 203, 208, 216, 223, 231, 252, 253, 260, 276, 277, 288, 292, 293, 296, 312, 316, 317, 318, 327, 329, 333, 339, 344, 353, 355, 356, 357, 364, 366, 367, 368, 370, 374, 377, 379, 380, 383, 393, 407, 410, 420, 423, 429, 430, 432, 435, 441, 442, 448, 455, 457, 466, 471, 473, 480, 500, ...]"
608,609,"[1, 10, 110, 116, 137, 150, 161, 185, 208, 231, 253, 288, 292, 296, 318, 329, 339, 356, 434, 454, 457, 480, 589, 590, 592, 613, 650, 731, 742, 786, 828, 833, 892, 1056, 1059, 1150, 1161]"


In [192]:
class AssociationRuleRecommender:
    def __init__(self, association_rules_df, grouped_df):
        self.association_rules_df = association_rules_df
        self.grouped_df = grouped_df
        
    def _get_recommendations_by_rules(self, user_row, topn=10):
        user_id = user_row['userId']
        movie_ids = str(user_row['movieId'])
        
        user_recommendations = {}
        for _, rule in self.association_rules_df.iterrows():
            antecedents = list(rule['antecedents'])
            consequents = list(rule['consequents'])
            confidence = rule['confidence']

            if all(item in movie_ids for item in antecedents):
                for consequent in consequents:
                    if consequent not in movie_ids:  # Check if consequent is not already watched by the user
                        if consequent not in user_recommendations:
                            user_recommendations[consequent] = []
                        user_recommendations[consequent].append(confidence)

        user_recommendations = {movie_id: round(sum(confidences) / len(confidences), 2) for movie_id, confidences in user_recommendations.items()}
        sorted_recommendations = sorted(user_recommendations.items(), key=lambda x: x[1], reverse=True)[:topn]

        return {'user_id': user_id, 'recommendations': sorted_recommendations}
    
    def recommend_items(self, user_id, topn=10):
        user_data = self.grouped_df[self.grouped_df['userId'] == user_id]
        recommendation = self._get_recommendations_by_rules(user_data.iloc[0], topn)
        
        return recommendation


/opt/anaconda3/envs/CIS242/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [193]:
association_rule_recommender_model = AssociationRuleRecommender(df_ar, grouped_df)
user_id = 123
recommendations = association_rule_recommender_model.recommend_items(user_id, topn=10)
print(recommendations)

{'user_id': 123, 'recommendations': [('225', 0.73), ('898', 0.73), ('258', 0.72), ('1939', 0.7), ('278', 0.69), ('98', 0.68), ('511', 0.67), ('508', 0.66), ('124', 0.65), ('900', 0.64)]}


/opt/anaconda3/envs/CIS242/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
class AssociationRuleRecommender:
    def __init__(self, association_rules_df, grouped_df, movie_df):
        self.association_rules_df = association_rules_df
        self.grouped_df = grouped_df
        self.movie_df = movie_df
        
    def _get_recommendations_by_rules(self, user_row, topn=10):
        user_id = user_row['userId']
        movie_ids = str(user_row['movieId'])

        user_recommendations = {}
        for _, rule in self.association_rules_df.iterrows():
            antecedents = list(rule['antecedents'])
            consequents = list(rule['consequents'])
            confidence = rule['confidence']

            if all(item in movie_ids for item in antecedents):
                for consequent in consequents:
                    if consequent not in movie_ids:  # Check if consequent is not already watched by the user
                        if consequent not in user_recommendations:
                            user_recommendations[consequent] = []
                        user_recommendations[consequent].append(confidence)

        user_recommendations_with_titles = []
        for movie_id, confidences in user_recommendations.items():
            matching_rows = self.movie_df[self.movie_df['movieId'] == int(movie_id)]
            if not matching_rows.empty:
                title = matching_rows['title'].values[0]
                average_confidence = round(sum(confidences) / len(confidences), 2)
                user_recommendations_with_titles.append((title, movie_id, average_confidence))

        sorted_recommendations = sorted(user_recommendations_with_titles, key=lambda x: x[2], reverse=True)[:topn]

        return {'user_id': user_id, 'recommendations': sorted_recommendations}
    
    def recommend_items(self, user_id, topn=10):
        user_data = self.grouped_df[self.grouped_df['userId'] == user_id]
        recommendation = self._get_recommendations_by_rules(user_data.iloc[0], topn)
        
        return recommendation

/opt/anaconda3/envs/CIS242/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
association_rule_recommender_model = AssociationRuleRecommender(df_ar, grouped_df,movies)
user_id = 123
recommendations = association_rule_recommender_model.recommend_items(user_id, topn=10)
recommendations

/opt/anaconda3/envs/CIS242/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'user_id': 123,
 'recommendations': [('Disclosure (1994)', '225', 0.73),
  ('Philadelphia Story, The (1940)', '898', 0.73),
  ("Kid in King Arthur's Court, A (1995)", '258', 0.72),
  ('Best Years of Our Lives, The (1946)', '1939', 0.7),
  ('Miami Rhapsody (1995)', '278', 0.69),
  ('Program, The (1993)', '511', 0.67),
  ('Philadelphia (1993)', '508', 0.66),
  ('American in Paris, An (1951)', '900', 0.64),
  ('Poetic Justice (1993)', '510', 0.63),
  ('Aristocats, The (1970)', '616', 0.62)]}